In [ ]:
from modules.utils import *
from sklearn.metrics import ndcg_score, roc_auc_score
import re

In [ ]:
concept_iri, iri2idx = load_concept_iri()
concept_embed = load_concept_embed()

## NN_rank random

In [ ]:
with open('result/nrc_nnrank_random.txt', 'w') as f:
    for fname in tqdm(os.listdir(refset_iri_dir)):
        refset_iri = np.loadtxt(refset_iri_dir + fname, dtype=str)
        refset_seed_iri = np.loadtxt(seed_random_iri_dir + fname, dtype=str)
        refset_name = get_refset_name(fname)

        y_true = np.zeros(shape=(len(concept_iri)))
        for item in refset_iri:
            y_true[iri2idx[item]] = 1.0

        for size in range(5):

            idxes = [iri2idx[i] for i in refset_seed_iri[:size+1]]

            rank = compute_rank_by_embed(idxes, concept_embed, concept_iri)
            y_pred = compute_sim_by_rank(rank, concept_iri)


            ndcg = ndcg_score([y_true], [y_pred])
            auc = roc_auc_score(y_true, y_pred)

            f.write("refset={:35s} seed_size={} NNRank score: NDCG={:.4f} AUC={:.4f}\n".format(refset_name, size+1, ndcg, auc))

## NN_rank select

In [ ]:
refset_iri_dir = 'data/SnomedCT/nrc_refset_iri/'
seed_select_iri_dir = 'data/SnomedCT/nrc_seed_select_iri/'

with open('result/nrc_nnrank_select.txt', 'w') as f:
    for fname in tqdm(os.listdir(refset_iri_dir)):
        refset_iri = np.loadtxt(refset_iri_dir + fname, dtype=str)
        refset_seed_iri = np.loadtxt(seed_select_iri_dir + fname, dtype=str)
        refset_name = get_refset_name(fname)

        y_true = np.zeros(shape=(len(concept_iri)))
        for item in refset_iri:
            y_true[iri2idx[item]] = 1.0

        for size in range(5):

            idxes = [iri2idx[i] for i in refset_seed_iri[:size+1]]

            rank = compute_rank_by_embed(idxes, concept_embed, concept_iri)
            y_pred = compute_sim_by_rank(rank, concept_iri)


            ndcg = ndcg_score([y_true], [y_pred])
            auc = roc_auc_score(y_true, y_pred)
            f.write("refset={:35s} seed_size={} NNRank score: NDCG={:.4f} AUC={:.4f}\n".format(refset_name, size+1, ndcg, auc))
            f.flush()

## Module random

In [ ]:
refset_iri_dir = 'data/SnomedCT/nhs_refset_iri/'
module_iri_path = 'data/SnomedCT/nhs_output_random_module_iri/'

with open('result/nhs_module_random.txt', 'w') as f:
    for fname in tqdm(os.listdir(refset_iri_dir)):
        refset_iri = np.loadtxt(refset_iri_dir + fname, dtype=str)
        refset_name = re.split('-\d*\.txt', fname)[0]#get_refset_name(fname)

        y_true = np.zeros(shape=(len(concept_iri)))
        for item in refset_iri:
            y_true[iri2idx[item]] = 1.0

        for fname in os.listdir(module_iri_path):
            size = re.findall('#(.*?).t', fname)[0]
            if refset_name in fname:
                y_pred = load_selection_result(module_iri_path + fname, iri2idx, len(concept_iri))
                ndcg = ndcg_score([y_true], [y_pred])
                auc = roc_auc_score(y_true, y_pred)

                f.write("refset={:35s} seed_size={} module score: NDCG={:.4f} AUC={:.4f}\n".format(refset_name, size, ndcg, auc))
                f.flush()

## Module select

In [ ]:
module_iri_path = 'data/SnomedCT/output_select_module_iri/'

with open('result/module_select.txt', 'w') as f:
    for fname in tqdm(os.listdir(refset_iri_dir))):
        refset_iri = np.loadtxt(refset_iri_dir + fname, dtype=str)
        refset_name = get_refset_name(fname)

        y_true = np.zeros(shape=(len(concept_iri)))
        for item in refset_iri:
            y_true[iri2idx[item]] = 1.0

        for fname in os.listdir(module_iri_path):
            size = re.findall('1#(.*?).t', fname)[0]
            if refset_name in fname:
                y_pred = load_selection_result(module_iri_path + fname, iri2idx, len(concept_iri))
                ndcg = ndcg_score([y_true], [y_pred])
                auc = roc_auc_score(y_true, y_pred)

                f.write("refset={:35s} seed_size={} module score: NDCG={:.4f} AUC={:.4f}\n".format(refset_name, size, ndcg, auc))

## Sig extension select

In [ ]:
refset_iri_dir = 'data/SnomedCT/nrc_refset_iri/'
sigext_iri_path_fmt = 'nrc_seed_select_iri_extended'
dir_path = 'data/SnomedCT/'

with open('result/nrc_sigext_select.txt', 'w') as f:
    for dirname in os.listdir(dir_path):
        if sigext_iri_path_fmt in dirname:
            K = dirname.split('_')[-1]
            depth = dirname.split('_')[-2]
            
            result_dir = dir_path + dirname + '/'
            
            for fname in os.listdir(refset_iri_dir):

                refset_iri = np.loadtxt(refset_iri_dir + fname, dtype=str)
                refset_name = re.split('-\d*\.txt', fname)[0]#get_refset_name(fname)

                y_true = np.zeros(shape=(len(concept_iri)))
                for item in refset_iri:
                    y_true[iri2idx[item]] = 1.0

                y_pred = load_selection_result(result_dir + fname, iri2idx, len(concept_iri))
                ndcg = ndcg_score([y_true], [y_pred])
                auc = roc_auc_score(y_true, y_pred)

                f.write("refset={:35s} __depth__={} seed_size={} sigext score: NDCG={:.4f} AUC={:.4f}\n".format(refset_name, depth, K, ndcg, auc))